In [1]:
from langchain_core.documents import Document
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

documents_to_load = [
    "../../test.pdf"
]

def load_documents(documents_to_load: List[str]) -> List[Document]:
    documents = []
    for doc in documents_to_load:
        if doc.endswith(".pdf"):
            loader = PyPDFLoader(doc)
        else:
            raise ValueError(f"Unsupported file type: {doc}")
        documents.extend(loader.load())
    return documents

documents = load_documents(documents_to_load)

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(documents)

In [2]:
from langchain_qdrant import QdrantVectorStore
from langchain_ollama import OllamaEmbeddings

# Create a vector store on QDrant (uses split chunks for proper RAG retrieval)
# Uses collection "rag_documents" - nomic-embed-text produces 768-dim embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_store = QdrantVectorStore.from_texts(
    texts=[doc.page_content for doc in splits],
    embedding=embeddings,
    metadatas=[doc.metadata for doc in splits],
    url="http://localhost:6333",
    collection_name="rag_documents",
)

In [20]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from pydantic import BaseModel, Field
from typing import Annotated, List, Literal
import operator

# Create the retriever to retrieve context from the vector store
retriever = vector_store.as_retriever()

# Create the model
llm = ChatOllama(model="llama3.2")

# Create structure
class ChatBot(BaseModel):
    chat_history: Annotated[List[BaseMessage], operator.add]
    question: str = Field(description="The question to be answered.")
    answer: str = Field(description="The answer to the question based on the context provided.")
    chat_ended: Literal["Yes", "No"] = Field(description="Whether the chat has ended.")

structured_llm = llm.with_structured_output(ChatBot)

# Create the template (use {context}, {question}, and {chat_history} as placeholders)
template = """
Answer the question based on the following context and recent conversation history.

Context from documents:
{context}

Recent conversation history:
{chat_history}

Current question: {question}

Answer:
"""

# Create the prompt
prompt = ChatPromptTemplate.from_template(template)

# Helper function to format chat history as a string
def format_chat_history(messages: List[BaseMessage]) -> str:
    """Format chat history messages into a readable string."""
    if not messages:
        return "No previous conversation."
    
    formatted = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            formatted.append(f"User: {msg.content}")
        elif isinstance(msg, AIMessage):
            formatted.append(f"Assistant: {msg.content}")
        else:
            formatted.append(f"{msg.type}: {msg.content}")
    
    return "\n".join(formatted)

# Create the rag chain with conversation history
def create_rag_chain_with_history(chat_history: ChatMessageHistory):
    """Create a RAG chain that includes conversation history."""
    def get_chat_history_string(inputs: dict) -> dict:
        # Get recent messages (last 10 messages to avoid token limits)
        recent_messages = chat_history.messages[-10:] if len(chat_history.messages) > 10 else chat_history.messages
        return {
            **inputs,
            "chat_history": format_chat_history(recent_messages)
        }
    
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | get_chat_history_string
        | prompt
        | structured_llm
    )
    
    return rag_chain

# Initialize conversation history
conversation_history = ChatMessageHistory()

# Create the rag chain with history support
rag_chain = create_rag_chain_with_history(conversation_history)

# Test the chain
result = rag_chain.invoke("What is the main idea of the document?")
print(result)

# Add the interaction to conversation history
conversation_history.add_user_message("What is the main idea of the document?")
conversation_history.add_ai_message(result.answer)

ChatBot(chat_history=[BaseMessage(content='The main idea of the document appears to be an award ceremony, specifically a scholarship awarded by A*STAR and the University of Southampton for doctoral studies in the UK and Singapore.', additional_kwargs={}, response_metadata={}, type='document_summary', score=0.745)], question='What is the main idea of the document?', answer='Awards 2019 - 2023| P H D S SCHOLARSHIP | A*STAR AND THE UNIVERSITY OF SOUTHAMPTON', chat_ended='No')